In [26]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

def rk4(func, y0, x):
    y0 = np.array(y0)
    n = len(x)
    f = lambda xi,yi: np.array(func(yi,xi))
    y = np.zeros( (n,) + y0.shape)
    h = (x[n-1] - x[0])/(n-1) 
    xi = x[0]
    y[0] = yi = y0[:]
    for i in range(1, n):
        k1 = h * f(xi, yi)
        k2 = h * f(xi + 0.5 * h, yi + 0.5 * k1)
        k3 = h * f(xi + 0.5 * h, yi + 0.5 * k2)
        k4 = h * f(xi + h, yi + k3)
        xi = x[i]
        y[i] = yi = yi + (k1 + 2*(k2 + k3) + k4) / 6
    return y

def bisect(z, del_z, zc):
    global lp, lm, z_minus, z_plus
    if zc > 0.:
        lp = 1
        z_plus = z
    else:
        lm = 1
        z_minus = z
    if lp*lm == 0:
        z += (lm - lp)*del_z
        return(z)
    else:
        z = 0.5*(z_plus + z_minus)
        return z
    
def potential(x):
    if x>= -1. and x<= 1.: return -1.
    else: return 0

def derivs(y0, x):

    dydx = np.zeros_like(y0)
    dydx[0] = y0[1]
    dydx[1] = lam*(potential(x)-E)*y0[0]

    return dydx


n_steps = 10000
tol = 1.e-5
x0 = -1.3 # x=0
xn = -x0 # x=1m

coeff_lam = 0.1
lam = coeff_lam*np.pi*np.pi #람다값
n_lam = str(coeff_lam)


E_i = -0.9999
a_asymp = 0.02 #점근거동에서 앞의 상수 (시행착오를 통해 결정)
del_E = 0.03#에너지 증가값

yn = 0 # An exact value of y at right point.
# create a x array from x0 to xn.
x = np.linspace(x0, xn, n_steps+1)

E = E_i
E_all = []

for nth in range(100): #여기서는 쏘아보기 법으로 에너지의 값만 구해줄거임. 에너지는 연산 과정에서만 존재하고, 따로 저장은 해놓지 않음.
    del_k = (-1)**(nth+1) * 0.5
    
    #global variable set for function bisect()
    loop = lp = lm = 0
    z_plus = z_ninus = 0.
    yc = 100000
    E_old = -100000
    
    while np.fabs(yc)>tol and np.fabs(E_old - E) > 1.e-100  and E < 0: #np.fabs는 절댓값을 반환하라 / 세 조건중에 하나라도 틀리면 바로 브레이크
        y00 = a_asymp*lam*lam*np.fabs(E)*np.exp(np.sqrt(lam*np.fabs(E))*x0)
        y01 = np.sqrt(lam*np.fabs(E))*y00
        y0 = [y00, y01]
        y = rk4(derivs, y0, x)
        yc = ((-1)**nth)*(((-1)**nth)*y00 - y[-1,0])/y00
        E= bisect(E, del_E, yc)
        loop += 1
        print('%dth bound state, loop = %d,  E = %.5f pi, yc = %.5e' % (nth+1, loop, E, yc))
        if loop > 100000: break #루프탈출 문
#    input()
#    print('\n')
    E_all.append(E) # append all k's to k_all #구한 k번째 에너지를 배열에 따로 저장.
    E = E + np.fabs(del_E) #다음 에너지를 구하기 위해 세팅해주는 초기값을, 바로 전단계 E보다 조금 더 큰 값에서 시작하자.
    if E > 0: break # n 번째 구속상태까지 계산하고 n+1번째를 계산하려고 했더니 E가 0보다 크네? 구속상태 없는것이 된다.

fig = plt.figure(figsize=(12,8))
ax = plt.gca()
plt.xlim(-3,3)
plt.ylim(-1.5, 1.5)
plt.title(r'Wave functions of time-independent Schr$\..{o}$dinger equation for a potential well case.')

for iE in range(len(E_all)-1): #위에서 구한 에너지의 값에서, 실제 y값들을 구하는 과정. 
    E = E_all[iE]
    y = rk4(derivs, y0, x)
    y[:,0] /= np.sqrt(integrate.simps(y[:,0]*y[:,0],x)) # 솔루션의 규격화 과정
    plt.text(0.7, 0.95-iE*0.04, r'%dth bound state energy = %.5f $U_0$' % (iE+1, E), transform=ax.transAxes)    
    line, = ax.plot(x,y[:,0], lw=0.8)
    ax.plot([x0, -1, -1, 1, 1, xn], [0, 0, -1, -1, 0, 0 ], 'b', lw=5, alpha=0.05)
plt.grid()
print(E_all)
#plt.savefig('square_potential_well_%s.png' %(n_lam))
plt.show()

1th bound state, loop = 1,  E = -0.96990 pi, yc = -3.61066e+00
1th bound state, loop = 2,  E = -0.93990 pi, yc = -3.35332e+00
1th bound state, loop = 3,  E = -0.90990 pi, yc = -3.10367e+00
1th bound state, loop = 4,  E = -0.87990 pi, yc = -2.86158e+00
1th bound state, loop = 5,  E = -0.84990 pi, yc = -2.62689e+00
1th bound state, loop = 6,  E = -0.81990 pi, yc = -2.39947e+00
1th bound state, loop = 7,  E = -0.78990 pi, yc = -2.17917e+00
1th bound state, loop = 8,  E = -0.75990 pi, yc = -1.96586e+00
1th bound state, loop = 9,  E = -0.72990 pi, yc = -1.75941e+00
1th bound state, loop = 10,  E = -0.69990 pi, yc = -1.55966e+00
1th bound state, loop = 11,  E = -0.66990 pi, yc = -1.36650e+00
1th bound state, loop = 12,  E = -0.63990 pi, yc = -1.17979e+00
1th bound state, loop = 13,  E = -0.60990 pi, yc = -9.99405e-01
1th bound state, loop = 14,  E = -0.57990 pi, yc = -8.25206e-01
1th bound state, loop = 15,  E = -0.54990 pi, yc = -6.57071e-01
1th bound state, loop = 16,  E = -0.51990 pi, yc 

KeyboardInterrupt: 